In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE

In [ ]:
ss = StandardScaler()
mm = MinMaxScaler()

In [ ]:
### load df containing radiomics feature and clinical feature 
X = df.basename.unique()
loo = LeaveOneOut()
loo.get_n_splits(X)

In [ ]:
clinic= ['basename','GENDER','RACE_ETHNICITY','INSURANCE','FAMILY_HX_TC','BMI',
'BETHESDA','SIZE_LARGEST_US','VOL_US','OPERATION','SIZE_LARGEST_PATH','label']

In [ ]:
### build model on clinical features only
df_clinic = df[clinic]
df_clinic = df_clinic.drop_duplicates(subset='basename',keep='first')
df_clinic = df_clinic.reset_index(drop=True)
df_clinic = df_clinic[['GENDER','RACE_ETHNICITY','INSURANCE','FAMILY_HX_TC','BMI',
'BETHESDA','SIZE_LARGEST_US','VOL_US','OPERATION','SIZE_LARGEST_PATH','label']]

In [ ]:
## use clinical variable only
output = pd.DataFrame()
prob_list= []
label_list= []
name_list=[]
X_clinic = df_clinic.drop(columns=['label','basename'])
X_clinic_ss = pd.DataFrame(ss.fit_transform(X_clinic),columns = X_clinic.columns)
X_clinic_ss['label']=df_clinic['label']
X_clinic_ss['basename']=df_clinic['basename']
importance = []
for train_index, test_index in loo.split(X):
     df_train = X_clinic_ss[X_clinic_ss.basename.isin(X[train_index])]
     df_test = X_clinic_ss[X_clinic_ss.basename.isin(X[test_index])]
     name_list.extend(df_test['basename'])
     Y_train = df_train['label']
     X_train = df_train.drop(['label','basename'],axis=1)
     Y_test = df_test['label']
     X_test = df_test.drop(['label','basename'],axis=1)
     #X_test_ss = pd.DataFrame(ss.fit_transform(X_test),columns = X_test.columns)
     #X_train_ss = pd.DataFrame(ss.fit_transform(X_train),columns = X_train.columns)

     rf = RandomForestClassifier(random_state=11)
     lr =LogisticRegression()
     
     model = rf.fit(X_train, Y_train)
     prob = model.predict_proba(X_test)
     prob_list.extend(prob[:,1])
     label_list.extend(Y_test)
     importance.append(rf.feature_importances_)
print(roc_auc_score(label_list,prob_list))

### get random forest importance score

importances = np.mean(importance,axis=0)
import_std = np.std(importance, axis=0)

feature_list = ['GENDER','RACE ETHNICITY','INSURANCE','FAMILY HISTORY','BMI',
'BETHESDA','Size on US','Volume on US','OPERATION','Size on PATH']
df_import = pd.DataFrame()
df_import['feautures'] = feature_list
df_import['importance'] = importances
###output feature importance in clinicl only model

In [ ]:
### image only

rf = RandomForestClassifier(random_state=42)
###drop clinical features
df_image = df.iloc[:,:-10]
df_image = df_image.drop(columns=['filename'])

### find 2 tsne image features

X_image = df_image.drop(columns=['label','basename'])
X_image_ss = pd.DataFrame(ss.fit_transform(X_image),columns = X_image.columns)
tsne = TSNE(n_components=2,random_state=42)
x_tsne = tsne.fit_transform(X_image_ss)
x_tsne = pd.DataFrame(x_tsne)

x_tsne['label']=df_image['label']
x_tsne['basename']=df_image['basename']

prob_list=[]
label_list=[]
name_list=[]
for train_index, test_index in loo.split(X):
     df_train = x_tsne[x_tsne.basename.isin(X[train_index])]
     df_test = x_tsne[x_tsne.basename.isin(X[test_index])]
     name_list.extend(df_test.basename.tolist())
     Y_train = df_train['label']
     X_train = df_train.drop(['label','basename'],axis=1)
     Y_test = df_test['label']
     X_test = df_test.drop(['label','basename'],axis=1)
     #X_test_ss = pd.DataFrame(ss.fit_transform(X_test),columns = X_test.columns)
     #X_train_ss = pd.DataFrame(ss.fit_transform(X_train),columns = X_train.columns)
     
     model = rf.fit(X_train, Y_train)
     prob = model.predict_proba(X_test)
     prob_list.extend(prob[:,1])
     label_list.extend(Y_test)
output_image=pd.DataFrame()
output_image['label']=label_list
output_image['prob'] = prob_list
output_image['basename']= name_list
out_img = output_image.groupby('basename').mean()

print(roc_auc_score(out_img['label'],out_img['prob']))


In [ ]:
x_tsne = x_tsne.drop(columns='label')

### combine tsne and clinic
idx_list=[]
auc_list=[]
#for i in range(100):
i=89
rf = RandomForestClassifier(random_state=i)
x_combine = pd.merge(x_tsne,df_clinic, on='basename',how='left')
prob_list=[]
label_list=[]
name_list=[]
for train_index, test_index in loo.split(X):
     df_train = x_combine[x_combine.basename.isin(X[train_index])]
     df_test = x_combine[x_combine.basename.isin(X[test_index])]
     name_list.extend(df_test.basename.tolist())
     Y_train = df_train['label']
     X_train = df_train.drop(['label','basename'],axis=1)
     Y_test = df_test['label']
     X_test = df_test.drop(['label','basename'],axis=1)
     #X_test_ss = pd.DataFrame(ss.fit_transform(X_test),columns = X_test.columns)
     #X_train_ss = pd.DataFrame(ss.fit_transform(X_train),columns = X_train.columns)
     
     model = rf.fit(X_train, Y_train)
     prob = model.predict_proba(X_test)
     prob_list.extend(prob[:,1])
     label_list.extend(Y_test)
output_com=pd.DataFrame()
output_com['label']=label_list
output_com['prob'] = prob_list
output_com['basename']= name_list
out_com = output_com.groupby('basename').mean()

print(roc_auc_score(out_com['label'],out_com['prob']))
